<a href="https://colab.research.google.com/github/abishekabi/Volcanic-Eruption-Prediction/blob/main/predict_volcanic_eruptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions'

Mounted at /content/gdrive


In [2]:
%cd 'gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions'

/content/gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions


In [8]:
# !wget "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19059/1581061/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1603216878&Signature=Xcz3Ni8%2F7bb7rIyQlyeyfbeASoi78143MpaVMe3kyZXi3WKfxcW7kp%2F44CQYzgoE2FfwzGR343oh5QVkHTt9xTGjHufJZRxbiJ0allxJiMFJrdZaZLZP8UwcQGl4k7ZlyF%2Bc7lAcoYNI14LjW4GEUSTOYvWyIrJqWGsBPwYCcLaGLaHP4lKxF0W7M%2FIOlyIMASugpiRIVsuCkR8fNWT3k4vyRgSldLTlfI46fkNvOFIsOl8kU%2BeRVd8yuhBrA8EmY8QRR0%2FGl9wSCfyjQe8Or4nSWs5V5svpRrzbtl5gTeMcIi8emEAktsbg8lK3%2F%2BzpIMu4map2I1aXj1wAqMPnYA%3D%3D&response-content-disposition=attachment%3B+filename%3Dpredict-volcanic-eruptions-ingv-oe.zip"
# !unzip data.zip
# from google.colab import files
# files.upload() 
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json  # set permission
#!kaggle competitions download -c predict-volcanic-eruptions-ingv-oe 

In [18]:
!ls -lh 

total 9.5G
-rw------- 1 root root 9.5G Oct 17 18:06 data.zip
-rw------- 1 root root  36K Oct 19 16:01 predict-volcanic-eruptions.ipynb
-rw------- 1 root root  56K Oct 17 18:08 sample_submission.csv
drwx------ 2 root root 4.0K Oct 19 15:53 test
drwx------ 2 root root 4.0K Oct 19 16:00 train
-rw------- 1 root root  84K Oct  8 23:38 train.csv


In [19]:
!head train.csv

segment_id,time_to_eruption
1136037770,12262005
1969647810,32739612
1895879680,14965999
2068207140,26469720
192955606,31072429
1640671020,22264866
288840453,34952168
1162441568,9504818
1791400073,11719563


In [20]:
!head train/1136037770.csv

sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
220.0,339.0,-336.0,364.0,,492.0,-796.0,-423.0,-582.0,-40.0
178.0,221.0,-317.0,366.0,,525.0,-754.0,-415.0,-932.0,-305.0
151.0,42.0,-280.0,250.0,,463.0,-772.0,-229.0,-257.0,-59.0
162.0,-123.0,-243.0,288.0,,303.0,-899.0,212.0,-295.0,38.0
158.0,-287.0,-300.0,372.0,,169.0,-769.0,755.0,169.0,-239.0
136.0,-302.0,-308.0,242.0,,241.0,-510.0,888.0,282.0,-293.0
124.0,-131.0,-292.0,132.0,,234.0,-494.0,970.0,-326.0,-203.0
127.0,133.0,-208.0,14.0,,134.0,-748.0,700.0,237.0,-278.0
142.0,374.0,-81.0,-177.0,,-200.0,-678.0,402.0,664.0,-460.0


In [21]:
!head sample_submission.csv

segment_id,time_to_eruption
1000213997,0
100023368,0
1000488999,0
1001028887,0
1001857862,0
1003310246,0
1003545497,0
1004288653,0
10044968,0


In [22]:
!head test/1000213997.csv

sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
180.0,,295.0,-221.0,-81.0,379.0,240.0,-288.0,152.0,0.0
200.0,,199.0,-99.0,32.0,-226.0,241.0,-114.0,36.0,0.0
193.0,,113.0,17.0,164.0,52.0,294.0,27.0,-21.0,0.0
188.0,,26.0,125.0,274.0,-278.0,324.0,338.0,-22.0,0.0
177.0,,-48.0,183.0,355.0,-363.0,356.0,339.0,-32.0,0.0
192.0,,-123.0,206.0,414.0,338.0,322.0,493.0,-59.0,0.0
198.0,,-187.0,224.0,452.0,-3.0,257.0,360.0,-30.0,0.0
202.0,,-227.0,221.0,450.0,845.0,208.0,212.0,2.0,0.0
211.0,,-292.0,235.0,388.0,-422.0,158.0,92.0,-4.0,0.0


In [23]:
!ls

data.zip			  sample_submission.csv  train
predict-volcanic-eruptions.ipynb  test			 train.csv


# Data

In [24]:
import gc
import glob
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm import tqdm
import seaborn as sns

In [25]:
train = pd.read_csv("train.csv")
sample_submission = pd.read_csv("sample_submission.csv")


In [27]:
segment_csvs = glob.glob("train/*")
segment_csvs_test = glob.glob("test/*")

In [29]:
segment_csvs[0:5]

['train/1000015382.csv',
 'train/1000554676.csv',
 'train/1000745424.csv',
 'train/1001461087.csv',
 'train/1001732002.csv']

In [32]:
print("Number of files under train/ is:", len(segment_csvs))
print("Number of files under test/ is:", len(segment_csvs_test))

duplicated_segment_id = [segment_id for segment_id in [ test_segment_id.split("/")[-1] for test_segment_id in segment_csvs_test]
                         if (segment_id in [ train_segment_id.split("/")[-1] for train_segment_id in segment_csvs])]

print("[Duplicates] Segment ids both in train and test are:", duplicated_segment_id)

Number of files under train/ is: 4431
Number of files under test/ is: 4520
[Duplicates] Segment ids both in train and test are: []


## Load Data

In [33]:
train_379022420 = pd.read_csv("train/379022420.csv")
train_1002275321 = pd.read_csv("train/1002275321.csv")

In [34]:
train.head()

,segment_id,time_to_eruption
0,1136037770,12262005
1,1969647810,32739612
2,1895879680,14965999
3,2068207140,26469720
4,192955606,31072429


In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4431 entries, 0 to 4430
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   segment_id        4431 non-null   int64
 1   time_to_eruption  4431 non-null   int64
dtypes: int64(2)
memory usage: 69.4 KB
